In [1]:
import ast

import pandas as pd
import os
from shapely.geometry import LineString
import geopandas as gpd
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import shapely.wkt
import matplotlib as mpl
from matplotlib.ticker import ScalarFormatter
import matplotlib.colors as mplc
from mpl_toolkits.axes_grid1 import make_axes_locatable



import scipy.stats
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table
from adjustText import adjust_text

from pathlib import Path

## Read data

In [2]:
cwd = Path.cwd()
parent_dir = cwd.parent.parent

parent_dir

PosixPath('/home/veror/Desktop/Tweet2Geo')

In [3]:
# merge iso2 - iso3
# geodata dataset

AREA = 'US'

gdf = gpd.read_file(parent_dir / 'Data' / 'States_shapefile.shp')
# Molloweide projection
#gdf = gdf.to_crs("ESRI:54009")
main_crs = 'epsg:2163'
# other projection for the US
gdf = gdf.to_crs(main_crs)
# iso2 - iso3 dataset
states = pd.read_csv(parent_dir / 'Data' / 'US' / 'US states no wa.csv')

gdf = gdf.rename(columns={'State_Code': 'ISO 2'})
states = states.rename(columns={'code': 'ISO 2'})
# merge -> with GF and SU
df_country = pd.merge(gdf, states, on='ISO 2', how='right')
#df_country.to_csv('Useful documents/info_country tot SA.csv', index=False)


## Choroplets

## out-flows


In [5]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines

# Read data frame of flows and convert it into GeoDataFrame
flows_dir = parent_dir / 'Data' / 'US' / '0 fluxes 0.5 no wa/'
flows_files = os.listdir(flows_dir)

legend_fontsize = 18

def adjust_colorbar(cb):
    cb.ax.tick_params(labelsize=legend_fontsize + 4)
    cb.ax.yaxis.label.set_size(legend_fontsize + 6)

# Save directory
save_dir = parent_dir / 'Data analysis' / 'Choroplets' / 'US' / 'out_flows_0.5'
colorsource = 'firebrick'
n = 0

for file in flows_files:
    country = file[:2]

    if 1:
        df_flow = pd.read_csv(flows_dir / file)
        df_flow = df_flow.drop(columns=['ISO 2'])
        df_flow = df_flow.rename(columns={'code': 'ISO 2'})
        df_flow = df_flow[~(df_flow['ISO 2'] == country)]

        print(country)

        df_source = df_country[df_country['ISO 2'] == country]
        df_flow = pd.merge(df_country, df_flow, on='ISO 2', how='left')

        # Convert DataFrame to GeoDataFrame
        df_flow = gpd.GeoDataFrame(df_flow, geometry='geometry', crs='EPSG:3857')
        df_source = gpd.GeoDataFrame(df_source, geometry='geometry', crs='EPSG:3857')

        # Create figure and grid layout
        fig = plt.figure(figsize=(7, 6), dpi=250)
        gs = gridspec.GridSpec(1, 2, width_ratios=[0.95, 0.05], wspace=0.04)

        ax = fig.add_subplot(gs[0])
        cax = fig.add_subplot(gs[1])

        # Plot flow data
        flow_plot = df_flow.plot(
            column='flux_percentage',
            legend=True,
            missing_kwds={'color': 'lightgrey'},
            legend_kwds={'label': 'Tweets out-flows percentage [%]'},
            ax=ax,
            cmap='Blues',
            cax=cax,
            rasterized=True
        )
        df_flow.boundary.plot(color='black', ax=ax, linewidth=0.5)

        # Plot source country
        df_source.plot(color=colorsource, alpha=0.5, ax=ax)
        df_source.boundary.plot(color=colorsource, linewidth=0.5, ax=ax)

        # Custom legend for source
        legend_handles = [mlines.Line2D([0], [0], color=colorsource, lw=4, label=f'Source: {df_source.state.values[0]}')]
        ax.legend(handles=legend_handles, loc='lower left', fontsize=12, frameon=False)

        # Format axes
        ax.axis('off')
        ax.tick_params(axis='both', labelsize=10)

        # Format colorbar
        cax.set_ylabel('Tweets out-flows percentage [%]', rotation=90, fontsize=18)
        cax.tick_params(labelsize=16)

        # Save figure
        file = country + '_choroplet.pdf'
        fig.savefig(save_dir / file, dpi=250, bbox_inches='tight')
        plt.close()

    n += 1


LA
TN
DC
SD
CO
UT
NM
FL
VT
NV
TX
ND
AL
MO
MS
SC
NC
PA
OH
AR
ME
DE
NJ
MA
IN
MN
WY
NY
NH
OR
GA
RI
OK
KS
WV
MD
CT
MI
CA
AZ
KY
MT
VA
ID
WI
IA
NE
IL


## inflows

In [6]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines
import matplotlib.colors as mplc

# Directories
flows_dir = parent_dir / 'Data' / 'US' / '0 fluxes 0.5 no wa'
flows_files = os.listdir(flows_dir)

save_dir = parent_dir / 'Data analysis' / 'Choroplets' / 'US' / 'in_flows_0.5'
legend_fontsize = 18
colorsource = 'firebrick'

def adjust_colorbar(cb):
    cb.ax.tick_params(labelsize=legend_fontsize + 4)
    cb.ax.yaxis.label.set_size(legend_fontsize + 6)

n = 0
for file in flows_files:
    country = file[:2]

    if file not in ['AK_count_US.csv', 'HI_count_US.csv']:  # Skip Alaska and Hawaii
        print(country)

        # Collect all in-flows from other countries toward `country`
        df_in_flows = pd.DataFrame(columns=[])
        for file1 in flows_files:
            country1 = file1[:2]
            if country1 != country:
                df_flow1 = pd.read_csv(os.path.join(flows_dir, file1))
                df_flow1 = df_flow1.drop(columns=['ISO 2'])
                df_flow1 = df_flow1.rename(columns={'code': 'ISO 2'})
                df_flow1 = df_flow1[df_flow1['ISO 2'] == country].reset_index(drop=True)
                if not df_flow1.empty:
                    df_flow1.at[0, 'ISO 2'] = country1  # Set the origin ISO code
                    df_in_flows = pd.concat([df_in_flows, df_flow1], ignore_index=True)

        # Merge with country geometries
        df_source = df_country[df_country['ISO 2'] == country]
        df_in_flows_tot = pd.merge(df_country, df_in_flows, on='ISO 2', how='left')

        # Convert to GeoDataFrames
        df_in_flows_tot = gpd.GeoDataFrame(df_in_flows_tot, geometry='geometry', crs='EPSG:3857')
        df_source = gpd.GeoDataFrame(df_source, geometry='geometry', crs='EPSG:3857')

        fig = plt.figure(figsize=(7, 6), dpi=250)
        gs = gridspec.GridSpec(1, 2, width_ratios=[0.95, 0.05], wspace=0.04)

        ax = fig.add_subplot(gs[0])
        cax = fig.add_subplot(gs[1])

        # Plot choropleth
        flow_plot = df_in_flows_tot.plot(
            column='flux_percentage',
            legend=True,
            missing_kwds={'color': 'lightgrey'},
            legend_kwds={'label': 'Tweets in-flows percentage [%]'},
            ax=ax,
            cmap='Blues',
            cax=cax,
            rasterized=True
        )
        df_in_flows_tot.boundary.plot(color='black', ax=ax, linewidth=0.5)

        # Highlight the destination country
        df_source.plot(color=colorsource, alpha=0.5, ax=ax)
        df_source.boundary.plot(color=colorsource, linewidth=0.5, ax=ax)

        # Add custom legend
        legend_handles = [
            mlines.Line2D([0], [0], color=colorsource, lw=4, label=f'Target: {df_source.state.values[0]}')
        ]
        ax.legend(handles=legend_handles, loc='lower left', fontsize=12, frameon=False)

        # Format colorbar
        cax.set_ylabel('Tweets in-flows percentage [%]', rotation=90, fontsize=18)
        cax.tick_params(labelsize=16)

        # Clean axes
        ax.tick_params(axis='both', labelsize=10)
        ax.axis('off')

        # Save figure
        file = country + '_choroplet.pdf'
        fig.savefig(save_dir / file, dpi=250, bbox_inches='tight')
        plt.close()

    n += 1


LA
TN
DC
SD
CO
UT
NM
FL
VT
NV
TX
ND
AL
MO
MS
SC
NC
PA
OH
AR
ME
DE
NJ
MA
IN
MN
WY
NY
NH
OR
GA
RI
OK
KS
WV
MD
CT
MI
CA
AZ
KY
MT
VA
ID
WI
IA
NE
IL


## self loops

In [7]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.lines as mlines

# Directories
flows_dir = parent_dir / 'Data' / 'US' / '0 fluxes 0.5 no wa'
flows_files = os.listdir(flows_dir)

save_dir = parent_dir / 'Data analysis' / 'Choroplets' / 'US'

legend_fontsize = 18
colorsource = 'firebrick'

# Initialize container
df_self_loops = pd.DataFrame(columns=[])

# Load and process each flow file
for file in flows_files:
    df_flow = pd.read_csv(flows_dir / file)
    country = file[0:2]
    df_flow = df_flow.drop(columns=['ISO 2'])
    df_flow = df_flow.rename(columns={'code': 'ISO 2'})
    country_row = df_flow[df_flow['ISO 2'] == country]
    df_flow = pd.merge(df_country, country_row, on='ISO 2')
    df_self_loops = pd.concat([df_self_loops, df_flow], ignore_index=True)

# Define color range
vmin = df_self_loops['flux_percentage'].min()
vmax = df_self_loops['flux_percentage'].max()

# Convert to GeoDataFrame if not already
df_self_loops = gpd.GeoDataFrame(df_self_loops, geometry='geometry', crs='EPSG:3857')

# Plotting setup using gridspec
fig = plt.figure(figsize=(7, 6), dpi=250)
gs = gridspec.GridSpec(1, 2, width_ratios=[0.95, 0.05], wspace=0.04)

ax = fig.add_subplot(gs[0])
cax = fig.add_subplot(gs[1])

# Plot main choropleth
df_self_loops.plot(
    column='flux_percentage',
    ax=ax,
    legend=True,
    missing_kwds={'color': 'lightgrey'},
    legend_kwds={'label': 'Percentage of self-loops [%]'},
    vmin=vmin,
    vmax=vmax,
    cmap='GnBu',
    cax=cax,
    rasterized=True
)

# Add boundaries
df_self_loops.boundary.plot(color='black', linewidth=0.5, ax=ax)

# Final formatting
cax.set_ylabel('Tweets self-loops [%]', rotation=90, fontsize=18)
cax.tick_params(labelsize=16)
ax.axis('off')

# Save figure
fig.savefig(save_dir / 'self_loops_0.5_choroplet.pdf', dpi=250, bbox_inches='tight')
plt.close()
